In [19]:
import numpy as np
import tifffile as tiff
import random
import scipy.ndimage
from tqdm import tqdm

# Set deterministic seed
seed = 42
random.seed(seed)
np.random.seed(seed)

# Load dataset from a TIFF stack
def load_tiff_stack(path):
    """Loads a multi-frame TIFF stack into a NumPy array."""
    return tiff.imread(path)

# Save dataset to a TIFF stack
def save_tiff_stack(dataset, path):
    """Saves a 3D NumPy array as a multi-frame TIFF stack."""
    tiff.imwrite(path, dataset, photometric='minisblack')

# Custom Flip and Rotate Transformations
def mirror_3d(volume):
    """Applies horizontal and vertical mirroring to a 3D volume."""
    volume = np.flip(volume, axis=1)  # Flip along height (Y-axis)
    volume = np.flip(volume, axis=2)  # Flip along width (X-axis)
    return volume


def rotate_90_3d(volume):
    """Rotates a 3D volume by 90 degrees along the last two dimensions."""
    return np.rot90(volume, k=1, axes=(1, 2))  # Rotate in the (height, width) plane


# Shearing Crop (removes fixed % of height or width)
def shear_crop_3d(volume, crop_percent=0.3, crop_dim="height"):
    """Shears off a fixed percentage of height or width from a 3D volume."""
    d, h, w = volume.shape

    if crop_dim == "height":
        new_h = int(h * (1 - crop_percent))
        volume = volume[:, :new_h, :]
    elif crop_dim == "width":
        new_w = int(w * (1 - crop_percent))
        volume = volume[:, :, :new_w]
    else:
        raise ValueError("Crop dimension must be 'height' or 'width'")

    return volume


def random_erasing_3d(volume, num_cubes=5, min_size=50, max_size=75):
    """Erases random cubic regions in a 3D volume."""
    d, h, w = volume.shape
    np.random.seed(seed)

    for _ in range(num_cubes):
        size = np.random.randint(min_size, max_size + 1)
        x = np.random.randint(0, d - size + 1)
        y = np.random.randint(0, h - size + 1)
        z = np.random.randint(0, w - size + 1)
        volume[x:x+size, y:y+size, z:z+size] = 0

    return volume


# Custom 3D Scaling (Only Height or Width)
def random_scale_3d(volume, scale_factor=0.3, axis="height"):
    """Scales a 3D volume along height or width."""
    d, h, w = volume.shape

    if axis == "height":
        zoom_factors = (1, scale_factor, 1)
    elif axis == "width":
        zoom_factors = (1, 1, scale_factor)
    else:
        raise ValueError("Axis must be 'height' or 'width'")

    return scipy.ndimage.zoom(volume, zoom=zoom_factors, order=1)


# Apply augmentations
def augment_volume(volume, config):
    """Apply selected augmentations to a 3D volume."""
    if config.get("mirror", False):
        volume = mirror_3d(volume)
    if config.get("rotate", False):
        volume = rotate_90_3d(volume)
    if config.get("shear_crop", False):
        volume = shear_crop_3d(volume, crop_percent=0.3, crop_dim="height")
    if config.get("random_erasing", False):
        volume = random_erasing_3d(volume)
    if config.get("scale_3d", False):
        volume = random_scale_3d(volume, scale_factor=0.3, axis=config.get("scale_axis", "height"))

    return np.array(volume, dtype=np.uint16)


# Process and save augmented dataset
def process_and_save_tiff(input_path, output_path, config):
    """Loads a TIFF stack, applies augmentations, and saves it back."""
    volume = load_tiff_stack(input_path)
    augmented_volume = augment_volume(volume, config)
    save_tiff_stack(augmented_volume, output_path)

In [21]:
# Main execution
if __name__ == "__main__":
    # Load dataset (NumPy-based)
    tablet_path = r'd:\Darren\Files\database\tablet_dataset\grayscale\tiff\2_Tablet'
    tablet = load_tiff_stack(tablet_path)

    # Choose augmentations
    config = {
        "mirror": True,          # Apply left-right and top-bottom mirroring
        "rotate": False,          # Apply 90-degree rotation
        "shear_crop": False,      # Apply shear cropping
        "random_erasing": False,  # Apply random cube erasing
        "scale_3d": False,        # Apply random 3D scaling
        "scale_axis": "height"   # Scale along height (Y-axis) by default
    }

    # Save augmented dataset
    output_path = r'd:\Darren\Files\database\tablet_dataset\grayscale\tiff\2_Tablet_augmented'

    # Apply augmentations with tqdm progress tracking
    augmented_tablet = process_and_save_tiff(tablet_path, output_path, config)

PermissionError: [Errno 13] Permission denied: 'D:\\Darren\\Files\\database\\tablet_dataset\\grayscale\\tiff\\2_Tablet'